In [1]:
import os
import sys
# sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.dirname(os.path.abspath(__file__)))
from data import setup_env
from pinecone_text.sparse import BM25Encoder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from datetime import datetime
from gtts import gTTS
from playsound import playsound
import speech_recognition as sr


NameError: name '__file__' is not defined

In [2]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
import os
from dotenv import load_dotenv
from typing_extensions import Annotated
from langgraph.prebuilt import InjectedState
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import InjectedStore
from langchain_core.documents import Document
from typing import List, Tuple


load_dotenv('../../.env')

# @tool
# def get_context(
#     question: str,
#     state
# ) -> Tuple[str, List[Document]]:
#     """Provides a relevant context when the model does not have enough information to answer the question"""
#     return "angel fombellida"
# tools = [get_context]
# tool_node = ToolNode(tools)

# model = ChatOpenAI(
#         model=os.getenv('OPENAI_MODEL'),
#         api_key = os.getenv('OPENAI_API_KEY'),
#         temperature=0.7,
#         max_tokens=600,
#         top_p=0.6,
#         timeout=None,
#         max_retries=2
#     )

# checkpointer = MemorySaver()
# graph = create_react_agent(model, tools, checkpointer=checkpointer)

True

In [23]:
tool_node.invoke({"messages":[model.invoke("Donde trabaja angel fombellida?")]})

{'messages': [ToolMessage(content='Angel fombellida trabaja en metiora', name='get_context', tool_call_id='call_y6hlpsq6SaAV4mi5RrVUFIp6')]}

In [101]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import AIMessage, HumanMessage

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


@tool
def get_context(
    question: str,
    state: MessagesState
) -> Tuple[str, List[Document]]:
    """Provides a relevant context""" # when the model does not have enough information to answer the question"""
    return "angel fombellida"


def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)
tools = [get_context]
tool_node = ToolNode(tools)

model = ChatOpenAI(
        model=os.getenv('OPENAI_MODEL'),
        api_key = os.getenv('OPENAI_API_KEY'),
        temperature=0.7,
        max_tokens=600,
        top_p=0.6,
        timeout=None,
        max_retries=2
    ).bind_tools(tools)


# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile(checkpointer = MemorySaver())

In [104]:
app.invoke({"messages": HumanMessage(content="i need relevant context")}, config={"thread_id":1})

{'messages': [HumanMessage(content='i need relevant context', additional_kwargs={}, response_metadata={}, id='3b3870ac-8b1a-46c6-8fa4-149365ace905'),
  AIMessage(content='Could you please provide me with the specific question or topic you need relevant context for? This will help me assist you better.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 2983, 'total_tokens': 3010, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2816}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-c001afc6-b003-4c70-a3e7-3ad8b64f2669-0', usage_metadata={'input_tokens': 2983, 'output_tokens': 27, 'total_tokens': 3010, 'input_token_details': {'audio': 0, 'cache_read': 2816}, 'output_token_details': {'audio': 

In [11]:
from typing import Literal
import sys
sys.path.append(os.path.abspath('../../'))
from langgraph.graph import StateGraph, MessagesState, START, END
from pinecone_text.sparse import BM25Encoder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, RemoveMessage

from datetime import datetime

from src.d01_data.data import *

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

    
def cohere_rerank_texts(texts, query, co_client, rerank_model='rerank-v3.5', top_n=None, threshold_score=0.3, debug=False):
    """
    Re-ranks a list of texts based on their similarity to a given query using the Cohere re-ranking API.

    Args:
        texts (list of str): A list of texts to be re-ranked.
        query (str): The query used to rank the texts by relevance.
        co_client (CohereClient): An instance of the Cohere client for accessing the rerank method.

    Returns:
        list of tuples: A list of tuples where each tuple contains:
            - original_index (int): The original position of the text in the input list.
            - new_rank (int): The new ranking based on relevance to the query.
            - document (dict): The document object, containing the text.
            - relevance_score (float): The relevance score assigned by the re-ranking model.
    """
    documents = [{'text': doc} for doc in texts]
    ranked_results = co_client.rerank(
        query=query,
        documents=documents,
        top_n=top_n,  # Rank all documents
        model=rerank_model,  # Specify the re-ranking model,
    )
    tmp_len = len(ranked_results.results)
    ranked_results = [response for response in ranked_results.results if response.relevance_score >= threshold_score]
    if tmp_len != len(ranked_results) and debug:
        print(f'Se han eliminado {tmp_len - len(ranked_results)} chunks que no superaron el tresholds')
        
    # Create a list of tuples with original index, new rank, document, and relevance score
    ranked_results = [(idx, rerank.index + 1, documents[rerank.index]['text'], rerank.relevance_score) for idx, rerank in enumerate(ranked_results, 1)]
    
    return ranked_results

    
def hybrid_scale(dense, sparse, alpha: float):
    if alpha < 0 or alpha > 1:
        raise ValueError('Alpha must be between 0 and 1')
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    
    return hdense, hsparse
    
    
def hybrid_query(index, question, embedding_model, fitted_bm25, top_k, alpha, filter={}):
    sparse_vec = fitted_bm25.encode_documents([question])[0]
    dense_vec = embedding_model.embed_documents([question])[0]
    dense_vec, sparse_vec = hybrid_scale(dense_vec, sparse_vec, alpha)
    result = index.query(
        vector=dense_vec,
        sparse_vector=sparse_vec,
        filter=filter,
        top_k=top_k,
        include_metadata=True
    )
    
    return result


def get_context2(question):
    bm25 = BM25Encoder().default()
    bm25.load('bm25_values.json')
    print(f"{datetime.now()}: Hybrid query...")
    results = hybrid_query(index= setup_db("hackaton"),question=question,embedding_model=setup_embeddings(),fitted_bm25=bm25,top_k=10,alpha=0.7)

    results_str = []
    for match in results['matches']:
        results_str.append(match['metadata']['text']) 
    
    print(f"{datetime.now()}: Reranking...")
    reranked = cohere_rerank_texts(
        texts=results_str,
        query=question,
        co_client=setup_cohere(),
        rerank_model='rerank-v3.5',
        top_n=5,
        threshold_score=0.3
    )
    return '\n\n'.join([f'Ranking {i}:\n{text[2]}' for i, text in enumerate(reranked, 1)][::-1])


@tool
def get_context(
    question: str,
    state: MessagesState
) -> Tuple[str, List[Document]]:
    """Provides a relevant context when the model does not have enough information to answer the question"""
    if isinstance(state["messages"][-1], AIMessage):
        summary = state["messages"].pop(-1)
        state["messages"].insert(1,summary)
    questions = []
    # if isinstance(state["messages"][1], AIMessage):
    #     questions.append(state["messages"][1].content)
    # for x in state["messages"]:
    #     if isinstance(x, HumanMessage):
    #         questions.append(x.content)
    questions = [state["messages"][-1].content]
    context = get_context2('\n'.join(questions))
    question_with_context = f"""
        <pregunta>
            {state['messages'][-1].content}
        </pregunta>
        <contexto>
            Aquí tienes el contenido complementario que has de usar para responder la pregunta:
            {context}
        <\contexto>"""
    messages = state["messages"][:-1] + [HumanMessage(content=question_with_context)] + [RemoveMessage(id = state["messages"][-1].id)]
    
    return {"messages": messages}


tools = [get_context]
tool_node = ToolNode(tools)
model = ChatOpenAI(
        model=os.getenv('OPENAI_MODEL'),
        api_key = os.getenv('OPENAI_API_KEY'),
        temperature=0.7,
        max_tokens=600,
        top_p=0.6,
        timeout=None,
        max_retries=2
    ).bind_tools(tools)

workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile(checkpointer=MemorySaver())

In [28]:
# example with a single tool call
app.invoke({"messages": [("human", "no soy ciudadano de la UE ni tengo antecedentes penales. Que tipo de autorizacion debo solicitar?")]}, config={"thread_id":1})#, stream_mode="values"

{'messages': [HumanMessage(content='no soy ciudadano de la UE ni tengo antecedentes penales. Que tipo de autorizacion debo solicitar?', additional_kwargs={}, response_metadata={}, id='f4094683-212b-428f-915d-cb4eaaf88aed'),
  AIMessage(content='Para poder ayudarte mejor, necesitaría un poco más de información sobre el contexto en el que deseas solicitar una autorización. Por ejemplo, ¿estás interesado en trabajar, estudiar, residir o visitar un país de la Unión Europea? Dependiendo de tu situación y del país específico al que desees ir, los requisitos pueden variar.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 3011, 'total_tokens': 3079, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c'

In [15]:
app.invoke({"messages": [("human", "soy victima de terrorismo en mi pais de origen, que documentacion necesito presentar?")]}, config={"thread_id":1})#, stream_mode="values"

2025-02-14 13:05:04.102277: Hybrid query...
The index hackaton already exists
2025-02-14 13:05:08.256638: Reranking...


{'messages': [HumanMessage(content='ya he presentado el EX-10, que hago?', additional_kwargs={}, response_metadata={}, id='8ac7c63a-c41b-47df-b908-b7e877cfbf19'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sbsFUZtr41ZotpcMBfF6UUWC', 'function': {'arguments': '{"question":"ya he presentado el EX-10, que hago?","state":{"messages":[{"content":"ya he presentado el EX-10, que hago?","type":"human"}]}}', 'name': 'get_context'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 3002, 'total_tokens': 3052, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2944}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6b0c6292-52ee-45c3-80e0-99305f47c03a-0', tool_calls=[{